# Format Europe energy demand

Data are hourly, country aggregates, from ENTSOE: https://www.entsoe.eu/data/power-stats/

In [112]:
import pandas as pd

In [113]:
data_path = "/g/data/w42/dr6273/work/data/ENTSOE/"

In [114]:
hourly = pd.read_csv(
    data_path+"Monthly-hourly-load-values_2006-2015.csv",
    skiprows=3,
    header=0,
    parse_dates={'timestamp': ['Year','Month', "Day"]},
    index_col="timestamp",
    # usecols=["Date", "Time", "NSW1", "QLD1", "SA1", "TAS1", "VIC1"]
)

In [115]:
hourly.head().iloc[:,2:].sum(axis=1)

timestamp
2006-01-01    145451.0
2006-01-02    176630.0
2006-01-03    179083.0
2006-01-04    179265.0
2006-01-05    178663.0
dtype: float64

Sum over hours

In [116]:
daily = hourly[["Country", "Coverage ratio"]]

In [117]:
daily["Unscaled power (MWh/day)"] = hourly.iloc[:,2:].sum(axis=1)

/jobfs/91062726.gadi-pbs/ipykernel_3846508/65306476.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  daily["Unscaled power (MWh/day)"] = hourly.iloc[:,2:].sum(axis=1)


According to the data, "To get the real value, you need to scale them to 100% for coverage ratio less than 100."

In [118]:
def scale(ratio, value):
    """
    Scale the value to 100% coverage
    
    ratio: coverage ration (%)
    value: value to scale to 100
    """
    return value / (ratio / 100) 

In [119]:
daily["Power (MWh/day)"] = daily.apply(
    lambda x : scale(x['Coverage ratio'], x["Unscaled power (MWh/day)"]), axis=1
)

/jobfs/91062726.gadi-pbs/ipykernel_3846508/2149852256.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  daily["Power (MWh/day)"] = daily.apply(


In [120]:
daily = daily.drop("Unscaled power (MWh/day)", axis=1)
daily = daily.drop("Coverage ratio", axis=1)

Only keep the following countries: Austria, Belgium, Denmark, France, Germany, Ireland, Italy, Luxembourg, the Netherlands, Norway, Portugal, Spain, Sweden, Switzerland and the UK.

In [123]:
daily["Country"].unique()

array(['AT', 'BA', 'BE', 'BG', 'CH', 'CS', 'CY', 'CZ', 'DE', 'DK', 'DK_W',
       'EE', 'ES', 'FI', 'FR', 'GB', 'GR', 'HR', 'HU', 'IE', 'IS', 'IT',
       'LT', 'LU', 'LV', 'ME', 'MK', 'NI', 'NL', 'NO', 'PL', 'PT', 'RO',
       'RS', 'SE', 'SI', 'SK', 'UA_W'], dtype=object)

In [124]:
codes = ["AT", "BE", "DK", "FR", "DE", "IE", "IT", "LU", "NL", "NO", "PT", "ES", "SE", "CH", "GB"]
len(codes)

15

In [130]:
daily = daily.loc[daily["Country"].isin(codes)]

Sum over all countries

In [131]:
daily = daily.groupby(daily.index).sum()

/jobfs/91062726.gadi-pbs/ipykernel_3846508/1035893.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  daily = daily.groupby(daily.index).sum()


In [132]:
daily.head()

,Power (MWh/day)
timestamp,
2006-01-01,4.559343e+06
2006-01-02,5.558190e+06
2006-01-03,5.934828e+06
2006-01-04,6.043556e+06
2006-01-05,6.128658e+06


In [133]:
daily.to_csv(
    data_path+"EU_daily_demand_2006-2015.csv"
)